<a href="https://colab.research.google.com/github/Benji8bit/dpro_spark/blob/main/spark_hw_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Шаг 1: Импорт необходимых библиотек и создание Spark Session

In [2]:
pip install pyspark

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, sum, datediff, to_date, when

# Создаем Spark Session
spark = SparkSession.builder.appName("COVID-19 Analysis").getOrCreate()

### Шаг 2: Загрузка данных
У нас есть CSV-файл с данными о COVID-19. Загрузим его в DataFrame:

In [5]:
# Файл называется 'covid-data.csv' и находится в той же директории
df = spark.read.csv('covid-data.csv', header=True, inferSchema=True)

In [6]:
df.show(5)

+--------+---------+-----------+----------+-----------+---------+------------------+------------+----------+-------------------+-----------------------+---------------------+------------------------------+------------------------+----------------------+-------------------------------+-----------------+------------+------------------------+-------------+-------------------------+---------------------+---------------------------------+----------------------+----------------------------------+---------+-----------+------------------------+----------------------+------------------+-------------------------------+-------------+--------------+-----------+------------------+-----------------+-----------------------+----------------+-------------------------+------------------------------+-----------------------------+-----------------------------------+-------------------------------------+----------------+-----------+------------------+----------+-------------+-------------+--------------+--

In [ ]:
df.printSchema()

###Шаг 3: Подготовка данных
Отфильтруем датасет по дате

In [8]:
df.count()

82289

In [20]:
filtered_df = df.filter(col("date") == lit("2020-03-31"))
filtered_df.count()

189

###Шаг 4: Выбор 15 стран с наибольшим процентом переболевших

In [21]:
total_cases_top_15_df = filtered_df.sort(filtered_df.total_cases.desc()).limit(15)

In [22]:
total_cases_top_15_df.select("iso_code", "location", "population", "total_cases").show()

+--------+--------------+-------------+-----------+
|iso_code|      location|   population|total_cases|
+--------+--------------+-------------+-----------+
|OWID_WRL|         World|7.794798729E9|   875794.0|
|OWID_EUR|        Europe| 7.48680069E8|   465241.0|
|OWID_EUN|European Union|  4.4491906E8|   397713.0|
|OWID_NAM| North America| 5.92072204E8|   205321.0|
|     USA| United States| 3.31002647E8|   192301.0|
|OWID_ASI|          Asia|4.639847425E9|   179033.0|
|     ITA|         Italy|  6.0461828E7|   105792.0|
|     ESP|         Spain|  4.6754783E7|    95923.0|
|     CHN|         China|1.439323774E9|    82279.0|
|     DEU|       Germany|  8.3783945E7|    71808.0|
|     FRA|        France|  6.8147687E7|    52278.0|
|     IRN|          Iran|  8.3992953E7|    44605.0|
|     GBR|United Kingdom|  6.7886004E7|    38815.0|
|     CHE|   Switzerland|    8654618.0|    16605.0|
|OWID_SAM| South America| 4.30759772E8|    14492.0|
+--------+--------------+-------------+-----------+



In [23]:
cases_percent_df = total_cases_top_15_df.withColumn("cases_percent", (total_cases_top_15_df.total_cases * 100)/total_cases_top_15_df.population)

In [24]:
cases_percent_df = cases_percent_df.select("iso_code", "location", "cases_percent").orderBy(cases_percent_df.cases_percent.desc()).show(15)

+--------+--------------+--------------------+
|iso_code|      location|       cases_percent|
+--------+--------------+--------------------+
|     ESP|         Spain| 0.20516189755388234|
|     CHE|   Switzerland| 0.19186288753587968|
|     ITA|         Italy|  0.1749732078891164|
|OWID_EUN|European Union| 0.08938996679530879|
|     DEU|       Germany| 0.08570615766540952|
|     FRA|        France| 0.07671280171255115|
|OWID_EUR|        Europe| 0.06214149665042038|
|     USA| United States|0.058096514255367875|
|     GBR|United Kingdom| 0.05717673410265833|
|     IRN|          Iran|0.053105645660535356|
|OWID_NAM| North America|0.034678371761563055|
|OWID_WRL|         World|0.011235620449591213|
|     CHN|         China|0.005716503922626098|
|OWID_ASI|          Asia|0.003858596708058...|
|OWID_SAM| South America|0.003364288158273...|
+--------+--------------+--------------------+



###Шаг 5: Выбор 10 стран с максимальным зафиксированным количеством новых случаев за последнюю неделю марта 2021

In [ ]:
# Фильтруем данные за последнюю неделю марта 2021
df_march_21 = df.filter((col("date") >= "2021-03-23") & (col("date") <= "2021-03-31"))

In [ ]:
# Группируем данные по стране и подсчитываем новые случаи
new_cases = df_march_21.groupBy("iso_code", "location").agg(sum("new_cases").alias("total_new_cases"))

In [ ]:
# Находим страны с максимальным количеством новых случаев
top_10_countries = new_cases.orderBy(new_cases.total_new_cases.desc()).limit(10)

In [ ]:
# Выводим результат
top_10_countries.select("location", "total_new_cases").show()

+--------------+---------------+
|      location|total_new_cases|
+--------------+---------------+
|         World|      5204960.0|
|        Europe|      1909681.0|
|European Union|      1529997.0|
|          Asia|      1405330.0|
| South America|      1074115.0|
| North America|       711840.0|
|        Brazil|       701221.0|
| United States|       588863.0|
|         India|       534869.0|
|        France|       346158.0|
+--------------+---------------+



###Шаг 6: Изменение случаев относительно предыдущего дня в России за последнюю неделю марта 2021

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lag

In [ ]:
# Фильтруем данные за последнюю неделю марта 2021
df_march_21 = df.filter((col("date") >= "2021-03-23") & (col("date") <= "2021-03-31")).filter(col("iso_code") == "RUS")
df_march_21.show(5)

+--------+---------+--------+----------+-----------+---------+------------------+------------+----------+-------------------+-----------------------+---------------------+------------------------------+------------------------+----------------------+-------------------------------+-----------------+------------+------------------------+-------------+-------------------------+---------------------+---------------------------------+----------------------+----------------------------------+---------+------------+------------------------+----------------------+------------------+-------------------------------+-------------+--------------+---------------+------------------+-----------------+-----------------------+----------------+-------------------------+------------------------------+-----------------------------+-----------------------------------+-------------------------------------+----------------+-----------+------------------+----------+-------------+-------------+--------------+

In [ ]:
# Группируем данные по дате и подсчитываем новые случаи
daily_cases = df_march_21.groupBy("iso_code", "date").agg(sum("new_cases").alias("cases"))
daily_cases.show(5)

+--------+----------+------+
|iso_code|      date| cases|
+--------+----------+------+
|     RUS|2021-03-24|8769.0|
|     RUS|2021-03-28|8979.0|
|     RUS|2021-03-27|8783.0|
|     RUS|2021-03-30|8162.0|
|     RUS|2021-03-25|9128.0|
+--------+----------+------+
only showing top 5 rows



In [ ]:
# Преобразуем данные в DataFrame с колонками для каждого дня
daily_cases_df = daily_cases.withColumn("prev_day", lag(col("cases"), 1).over(Window.orderBy("date")))
daily_cases_df.show(5)

+--------+----------+------+--------+
|iso_code|      date| cases|prev_day|
+--------+----------+------+--------+
|     RUS|2021-03-23|8369.0|    NULL|
|     RUS|2021-03-24|8769.0|  8369.0|
|     RUS|2021-03-25|9128.0|  8769.0|
|     RUS|2021-03-26|9073.0|  9128.0|
|     RUS|2021-03-27|8783.0|  9073.0|
+--------+----------+------+--------+
only showing top 5 rows



In [ ]:
# Вычисляем изменение случаев
delta_cases_rus = daily_cases_df.withColumn("delta", col("cases") - col("prev_day"))
delta_cases_rus.show(5)

+--------+----------+------+--------+------+
|iso_code|      date| cases|prev_day| delta|
+--------+----------+------+--------+------+
|     RUS|2021-03-23|8369.0|    NULL|  NULL|
|     RUS|2021-03-24|8769.0|  8369.0| 400.0|
|     RUS|2021-03-25|9128.0|  8769.0| 359.0|
|     RUS|2021-03-26|9073.0|  9128.0| -55.0|
|     RUS|2021-03-27|8783.0|  9073.0|-290.0|
+--------+----------+------+--------+------+
only showing top 5 rows



In [ ]:
# Выводим результат
delta_cases_rus.select("date", "cases", "delta").orderBy("date").show()

+----------+------+------+
|      date| cases| delta|
+----------+------+------+
|2021-03-23|8369.0|  NULL|
|2021-03-24|8769.0| 400.0|
|2021-03-25|9128.0| 359.0|
|2021-03-26|9073.0| -55.0|
|2021-03-27|8783.0|-290.0|
|2021-03-28|8979.0| 196.0|
|2021-03-29|8589.0|-390.0|
|2021-03-30|8162.0|-427.0|
|2021-03-31|8156.0|  -6.0|
+----------+------+------+

